## Reachy's arms

In this notebook you will learn how to use Reachy's SDK to control Reachy's arms!

In [1]:
from reachy_sdk import ReachySDK

Next you need to connect to your Reachy. You will use *ReachySDK* for that.
The *ReachySDK* object takes three arguments at init, one of them is *host* (you don't need to worry about the others).

If you're working directly on Reachy, use 
```python
host='localhost'
```
If not, just get Reachy's IP address with
```bash
$ ifconfig
```
and put this addreess in the *host* argument.

In [2]:
reachy = ReachySDK(host='192.168.0.23')

You can use ReachySDK from any computer, as long as it and Reachy are connected to the same network. 

With this Reachy object, you can access each joints.

In [3]:
joints_names = [joint.name for joint in reachy.joints]
joints_names

['r_shoulder_pitch',
 'r_shoulder_roll',
 'r_arm_yaw',
 'r_elbow_pitch',
 'r_forearm_yaw',
 'r_wrist_pitch',
 'r_wrist_roll',
 'r_gripper',
 'l_shoulder_pitch',
 'l_shoulder_roll',
 'l_arm_yaw',
 'l_elbow_pitch',
 'l_forearm_yaw',
 'l_wrist_pitch',
 'l_wrist_roll',
 'l_gripper']

You can check the name of each joint on the robot on the documentation: 

<img src="images/arm_schematic.png" alt="drawing" width="400"/>

In [4]:
right_joints = reachy.joints[:8]
left_joints = reachy.joints[8:]

### Check motors' positions

Each motor's position can be checked with the *present_position* attribute. The value is given in degrees. 

In [5]:
for joint in reachy.joints:
    print(f'{joint.name} - position:{joint.present_position}')

r_shoulder_pitch - position:-1.252747300508547
r_shoulder_roll - position:-19.186812463529755
r_arm_yaw - position:47.78021950860902
r_elbow_pitch - position:-60.70329941829915
r_forearm_yaw - position:-8.064516197381714
r_wrist_pitch - position:-55.4285731336355
r_wrist_roll - position:-7.184751120017283
r_gripper - position:-29.472139910105376
l_shoulder_pitch - position:26.26373709466734
l_shoulder_roll - position:1.252747300508547
l_arm_yaw - position:-5.846153997892082
l_elbow_pitch - position:-68.70329508745999
l_forearm_yaw - position:4.838709633051664
l_wrist_pitch - position:-48.571428622063294
l_wrist_roll - position:17.44867958392325
l_gripper - position:-15.689149429194389


Before developping applications with Reachy, you should check the max positions Reachy can attain without harming itself. 

For example if you put Reachy on a table, it cannot have its arms straight at by it side. 

### Compliant / Stiff mode

Each Reachy's motor can be in two modes: **compliant** and **stiff**.

In the compliant mode, you can move the motor freely with your hands. 

In stiff mode, the motor will be "blocked" (trouver un autre mot) and can only be move using the *goal_position* attribute of each joint. Keep that in mind, if a joint does not move when you're changing its *goal_position*, it's very likely that it is in compliant mode.

For example, to move the right elbow pitch joint at 90° just type: 
```python
reachy.r_elbow_pitch.compliant = False  #stiff mode
reachy.r_elbow_pitch.goal_position = 90
```

Once you execute
```python
reachy.r_elbow_pitch.compliant = False
```
the right elbow pitch joint will stay in stiff mode until you put it in compliant mode, or turn off the power. You don't need to write this line everytime you want to send a goal position to the joint.

You can easily give instructions to an arm once you know that. For example, to put the left arm at a right angle position you can do the following.

In [8]:
def arm_stiff(arm):
    for joint in arm:
        joint.compliant = False

right_angle_position = {
    'r_shoulder_roll': 0,
    'r_arm_yaw': 0,
    'r_elbow_pitch': -90,
    'r_forearm_yaw': 0,
    'r_wrist_pitch': 0,
    'r_wrist_roll': 0,
    'r_gripper': 0,
}

arm_stiff(right_joints)

In [9]:
from operator import attrgetter

In [10]:
for name, goal in zip(right_angle_position.keys(), right_angle_position.values()):
    joint = attrgetter(name)(reachy)
    joint.goal_position = goal

Don't forget to put the arm back into compliant mode! 

The motors will suffer if they are maintained in stiff mode in a position where the motor is sollicitated **(reformuler ça)**. Put your hand under the arm to prevent it from damaging when you turn it compliant.

In [11]:
def arm_compliant(arm):
    for joint in arm:
        joint.compliant = True

In [14]:
arm_compliant(right_joints)

### Check motors' temperatures

You also have access to each motors temperatures with the *temperature* attribute of each joint. 
Check the temperatures when you play with Reachy to avoid damaging it. You should take a break from Reachy if the temperatures are above 45° (à comfirmer). 

Anyway, the fans in each arm should turn on if the temperatures hit 45°.

In [17]:
for joint in reachy.joints:
    print(f'{joint.name} - temp:{joint.temperature}')

r_shoulder_pitch - temp:34.0
r_shoulder_roll - temp:35.0
r_arm_yaw - temp:35.0
r_elbow_pitch - temp:36.0
r_forearm_yaw - temp:31.0
r_wrist_pitch - temp:34.0
r_wrist_roll - temp:32.0
r_gripper - temp:32.0
l_shoulder_pitch - temp:33.0
l_shoulder_roll - temp:35.0
l_arm_yaw - temp:35.0
l_elbow_pitch - temp:34.0
l_forearm_yaw - temp:31.0
l_wrist_pitch - temp:33.0
l_wrist_roll - temp:32.0
l_gripper - temp:33.0


### Motor's speed

You can also change the speed limit of each motors with the *speed_limit* attribute of each joint. It can be useful if you want to add safety to Reachy's movements.

According to the motors documentation, the speed is in rpm (raw per minute).

<img src="images/dynamixel_speed.png" alt="drawing" width="600"/>



source: https://emanual.robotis.com/docs/en/dxl/ax/ax-18a/#ccw-angle-limit

By default, we set each motor's speed to 0, meaning the maximum rpm of the motor is used without controlling the speed.

For example if you want to set the speed limit of the right gripper to 300, just do the following:

In [47]:
reachy.r_gripper.speed_limit = 300